In [1]:
import pandas as pd

# Read the Parquet file into a DataFrame
df_temp = pd.read_parquet('806016_temp.parquet')

# Display the first few rows
df_temp.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '806016_temp.parquet'

In [ ]:
# Read the Parquet file into a DataFrame
df_events = pd.read_parquet('806016_events.parquet')

# Display the first few rows
df_events.head(20)

In [ ]:
import pandas as pd

Rows_remove = 100

# Load your Parquet file
df = pd.read_parquet('806016_temp.parquet')

# Identify the indices where an event has occurred
event_indices = df[df['Event'].notnull()].index

# Create sets to store indices for rows to collect and keep
indices_to_collect = set()
for idx in event_indices:
    # Add a range of indices around each event
    start_idx = max(0, idx - Rows_remove)
    end_idx = min(len(df), idx + Rows_remove)
    indices_to_collect.update(range(start_idx, end_idx + 1))

# Convert to list and sort indices (just for clarity)
indices_to_collect = sorted(indices_to_collect)

# Extract rows to collect into a separate DataFrame
df_around_events = df.iloc[indices_to_collect]

# Drop these rows from the original DataFrame to get the cleaned data
df_cleaned = df.drop(indices_to_collect)

# Save both DataFrames
df_around_events.to_parquet('around_events_data.parquet', index=False)
df_cleaned.to_parquet('cleaned_data.parquet', index=False)

In [ ]:
# Read the Parquet file into a DataFrame
df_temp = pd.read_parquet('around_events_data.parquet')

pd.set_option('display.max_rows', None)

# Display the first few rows
df_temp.head(25)

In [ ]:
import numpy as np

# Parameters
Rows_remove = 100
start_date = '2012-03-30'
end_date = '2012-12-01'

# Read the cleaned Parquet file into a DataFrame
df = pd.read_parquet('cleaned_data.parquet')

# Convert 'Datetime' to datetime type if it's not already
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set 'Datetime' as the index for easier plotting
df.set_index('Datetime', inplace=True)

# Filter the DataFrame to the specified date range
df_filtered = df.loc[start_date:end_date]

# Select only numeric columns for plotting
numeric_columns = df_filtered.select_dtypes(include=['float64', 'int64']).columns

# Plot each numeric column separately
for col in numeric_columns:
    plt.figure(figsize=(12, 6))
    plt.plot(df_filtered.index, df_filtered[col], label=col, color='blue')

    # Highlight gaps caused by the removal of rows around events
    removed_indices = np.where(df_filtered[col].isnull())[0]
    if len(removed_indices) > 0:
        for idx in removed_indices:
            plt.axvline(df_filtered.index[idx], color='red', linestyle='--', alpha=0.5)

    plt.title(f"Time Series of {col} (Filtered: {start_date} to {end_date})")
    plt.xlabel("Datetime")
    plt.ylabel(f"{col} Value")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.legend()
    plt.show()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Parameters
Rows_remove = 1000
start_date = '2012-03-30'
end_date = '2012-12-01'

# Read the cleaned Parquet file into a DataFrame
df = pd.read_parquet('cleaned_data.parquet')

# Convert 'Datetime' to datetime type if it's not already
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set 'Datetime' as the index for easier plotting
df.set_index('Datetime', inplace=True)

# Filter the DataFrame to the specified date range
df_filtered = df.loc[start_date:end_date]

# Select only numeric columns for plotting
numeric_columns = df_filtered.select_dtypes(include=['float64', 'int64']).columns

# Calculate time differences between consecutive rows
time_diffs = df_filtered.index.to_series().diff().dt.total_seconds()

# Define a threshold (e.g., 2x the median difference) to detect large gaps
gap_threshold = 2 * time_diffs.median()
gap_indices = time_diffs[time_diffs > gap_threshold].index

# Plot each numeric column separately
for col in numeric_columns:
    plt.figure(figsize=(12, 6))
    plt.plot(df_filtered.index, df_filtered[col], label=col, color='blue')

    # Highlight gaps in the data where rows were removed
    for gap in gap_indices:
        plt.axvline(gap, color='red', linestyle='--', alpha=0.8)

    plt.title(f"Time Series of {col} (Filtered: {start_date} to {end_date})")
    plt.xlabel("Datetime")
    plt.ylabel(f"{col} Value")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.legend()
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_data.parquet'